In [68]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_score 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import clone
from sklearn import linear_model
#silence future warning message
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [69]:
df = pd.read_csv('sales_train.csv')

In [70]:
df = df.drop(['date'], axis = 1)

In [71]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.00,1.0
1,0,25,2552,899.00,1.0
2,0,25,2552,899.00,-1.0
3,0,25,2554,1709.05,1.0
4,0,25,2555,1099.00,1.0


In [72]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]

rows_with_NaN.head(50)

,date_block_num,shop_id,item_id,item_price,item_cnt_day


In [73]:
def overview(dataframe):
    
    data_resumen = {'Valores Unicos':dataframe.nunique(),'No-Nulos': dataframe.notnull().sum(), 'Nulos': dataframe.isnull().sum(), 'Formato': dataframe.dtypes, 'Min': dataframe.min(), 'Max': dataframe.max()}
    resumen = pd.DataFrame(data=data_resumen)
    return resumen

In [74]:
overview(df)

,Valores Unicos,No-Nulos,Nulos,Formato,Min,Max
date_block_num,34,2935849,0,int64,0.0,33.0
shop_id,60,2935849,0,int64,0.0,59.0
item_id,21807,2935849,0,int64,0.0,22169.0
item_price,19993,2935849,0,float64,-1.0,307980.0
item_cnt_day,198,2935849,0,float64,-22.0,2169.0


In [75]:
#Get rid of outliers
df = df[(df['item_price']<100000) & (df['item_price']>=0)]
df = df[(df['item_cnt_day']<1000) & (df['item_cnt_day']>=0)]

# plot after outliers removal
#plot_features = ['item_price','item_cnt_day']
#for f in plot_features:
#   df.boxplot(f,f)


In [76]:
df2 = df.groupby(by = ['date_block_num','shop_id','item_id','item_price'])['item_cnt_day'].sum()

In [80]:
df2 = df2.reset_index(drop=False)
df2['total_sales'] = df2['item_price'] * df2['item_cnt_day']
df2.head()

,index,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_sales,item_id_avg_item_price
0,0,0,0,32,221.0,6.0,1326.0,1326.0
1,1,0,0,33,347.0,3.0,1041.0,1041.0
2,2,0,0,35,247.0,1.0,247.0,247.0
3,3,0,0,43,221.0,1.0,221.0,221.0
4,4,0,0,51,127.0,1.0,127.0,128.5


In [78]:
df2['item_id_avg_item_price'] = df2.groupby(['date_block_num','shop_id','item_id'])['total_sales'].transform('mean')


In [93]:
df2.head(30)

,index,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_sales,item_id_avg_item_price
0,0,0,0,32,221.0,6.0,1326.0,1326.0
1,1,0,0,33,347.0,3.0,1041.0,1041.0
2,2,0,0,35,247.0,1.0,247.0,247.0
3,3,0,0,43,221.0,1.0,221.0,221.0
4,4,0,0,51,127.0,1.0,127.0,128.5
5,5,0,0,51,130.0,1.0,130.0,128.5
6,6,0,0,61,195.0,1.0,195.0,195.0
7,7,0,0,75,76.0,1.0,76.0,76.0
8,8,0,0,88,76.0,1.0,76.0,76.0
9,9,0,0,95,193.0,1.0,193.0,193.0


In [86]:
df3 = df2.groupby(by = ['date_block_num','shop_id','item_id'])['item_id_avg_item_price'].mean()

In [87]:
df3 = df3.reset_index(drop=False)
df3.head(6)

,date_block_num,shop_id,item_id,item_id_avg_item_price
0,0,0,32,1326.0
1,0,0,33,1041.0
2,0,0,35,247.0
3,0,0,43,221.0
4,0,0,51,128.5
5,0,0,61,195.0


In [90]:
df3['item_id_sum_item_cnt_day'] = df2.groupby(by = ['date_block_num','shop_id','item_id'])['item_cnt_day'].transform('sum')
df3['item_id_avg_item_cnt_day'] = df2.groupby(by = ['date_block_num','shop_id','item_id'])['item_cnt_day'].transform('mean')

In [94]:
df3.head()

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day
0,0,0,32,1326.0,6.0,6.0
1,0,0,33,1041.0,3.0,3.0
2,0,0,35,247.0,1.0,1.0
3,0,0,43,221.0,1.0,1.0
4,0,0,51,128.5,2.0,1.0


In [95]:
df3['shop_id_avg_item_price'] = df2.groupby(['date_block_num','shop_id'])['total_sales'].transform('mean')



In [103]:
df3.sample(10)

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price
1271584,24,27,12187,998.00,3.0,3.0,3356.047016
227397,3,47,3679,124.50,2.0,2.0,1267.279275
893272,16,21,10077,104.33,1.0,1.0,1255.746411
1218563,23,27,17221,149.00,3.0,3.0,1780.689139
420359,7,17,3320,1416.01,2.0,2.0,1148.876457
1336421,25,54,6405,299.00,1.0,1.0,3073.952822
1250691,23,57,6888,299.00,1.0,1.0,3648.193261
141435,2,23,15569,398.00,1.0,1.0,1274.209934
1323991,25,35,1389,9992.00,1.0,1.0,3091.805102
552393,9,43,3710,799.00,1.0,1.0,1614.390789
